In [1]:
import plotly.graph_objects as go
import networkx as nx
import dash
from dash import dcc
from dash import html
import json
from dash.dependencies import Input, Output
import math

In [2]:
def load_json(json_path= 'data.json'):
    with open(json_path, 'rb') as j:
        d = json.load(j)
    return d

In [3]:
def create_graph(data):
    G = nx.DiGraph()
    G.add_nodes_from(data["nodes"])
    edges = list(map(tuple, data["edges"]))
    G.add_edges_from(edges)
    return G

In [4]:
def create_edges_annots(graph, node_positions):  
    edges = []
    
    for edge in graph.edges():
        
        x0 = node_positions[edge[0]][0]
        y0 = node_positions[edge[0]][1]
        x1 = node_positions[edge[1]][0]
        y1 = node_positions[edge[1]][1]
                
        length = math.sqrt((x1-x0)**2 + (y1-y0)**2)
        dotSizeConversion = .0565/20
        convertedDotDiameter = 10 * dotSizeConversion
        lengthFracReduction = convertedDotDiameter / length
        lengthFrac = 0.9 - lengthFracReduction
        
        skipY = (y1-y0) * (1-lengthFrac)
        skipX = (x1-x0) * (1-lengthFrac)
        
        x0 = x0 + skipX/2
        x1 = x1 - skipX/2
        y0 = y0 + skipY/2
        y1 = y1 - skipY/2
        
        edges.append(
                dict(ax=x0, ay=y0, 
                     x=x1, y=y1,
                    axref='x', ayref='y',
                    showarrow=True, arrowhead=1,
                    arrowcolor="black", arrowsize=1.5,
                    arrowwidth=2))
        
    return edges 

In [5]:
def create_node_traces(graph,node_positions, data):    
    node_x_green = []
    node_y_green = []
    node_x_yellow = []
    node_y_yellow = []
    node_x_red = []
    node_y_red = []
    node_x_blue = []
    node_y_blue = []
    
    for ix, node in enumerate(graph.nodes()):
        if data["status"][ix] == "DONE":
            x, y = node_positions[node]
            node_x_green.append(x)
            node_y_green.append(y)
            
        elif data["status"][ix] == "PENDING":
            x, y = node_positions[node]
            node_x_yellow.append(x)
            node_y_yellow.append(y)
            
        elif data["status"][ix] == "FAILED":
            x, y = node_positions[node]
            node_x_red.append(x)
            node_y_red.append(y)
            
        elif data["status"][ix] == "RUNNING":
            x, y = node_positions[node]
            node_x_blue.append(x)
            node_y_blue.append(y)
            
    

    node_trace_green= go.Scatter(
        x=node_x_green, y=node_y_green,
        mode='markers',
        marker_symbol = "circle",
        hoverinfo='none',
        marker_size=60,
        marker_color="green",
        line_width=5,
        name = "Done"
    )
    
    node_trace_yellow = go.Scatter(
        x=node_x_yellow, y=node_y_yellow,
        mode='markers',
        hoverinfo='none',
        marker_symbol = "circle",
        marker_size=60,
        marker_color="yellow",
        line_width=5,
        name = "Pending"
    )
    
    node_trace_red = go.Scatter(
        x=node_x_red, y=node_y_red,
        mode='markers',
        hoverinfo='none',
        marker_symbol = "circle",
        marker_size=60,
        marker_color="red",
        line_width=5,
        name = "Failed"
    )
    node_trace_blue = go.Scatter(
        x=node_x_blue, y=node_y_blue,
        mode='markers',
        hoverinfo='none',
        marker_symbol = "circle",
        marker_size=60,
        marker_color="lightskyblue",
        line_width=5,
        name = "Running"
    )
        
    return [node_trace_yellow, node_trace_red, node_trace_green, node_trace_blue]

In [6]:
def create_node_annots(layout):
    names = []
    for i in layout.keys():
        names.append(
            dict(text = i, 
                showarrow = False,
                x = layout[i][0],
                y = layout[i][1],
                font = dict(
                    color = 'black',
                    size = 15
                )
            ) 
        )
    return names

In [7]:
app = dash.Dash()
data = load_json()


app.layout= html.Div([
    html.Div(children=[
        html.H1(
            children='CLS-Luigi Live Visualizer',
            style=dict(textAlign='center')
        )
    ]),
    html.Div([
        html.Label(['Pipeline'], style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(
            list(range(1, len(data['pipelines'])+1, 1)),
            1,
            id='pipeline_index',
        ),
    ]),
    dcc.Graph(id='pipelines_graph'),
    dcc.Interval(
        id='interval-component',
        interval=1*1000, 
        n_intervals=0
    )
])

@app.callback(
    Output('pipelines_graph', 'figure'),
    Input('pipeline_index', 'value'),
    Input('interval-component', 'n_intervals'))
def update_graph(pipeline_index, n):
    data = load_json()['pipelines'][pipeline_index-1]
    G = create_graph(data)
    node_positions = nx.planar_layout(G)
    
    node_traces = create_node_traces(G, node_positions, data)
    node_names = create_node_annots(node_positions)  
    edges = create_edges_annots(G, node_positions)
    
    annots = edges + node_names
    
    
    fig = go.Figure(
        data=node_traces,
        layout=go.Layout(
            annotations = annots,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
        )
    )
    colors = {
        'background': 'darkgrey'}
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
    )
    return fig
    
app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [11/Jul/2022 15:23:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:23:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:23:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:23:57] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 15:23:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 15:23:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jul/2022 15:23:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:23:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:23:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:24:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2022 15:24:01] "POST /_dash-update-component HTTP/1.1